In [1]:
import os
import glob2
from IPython.display import Image, display
import pandas as pd
import pipeline as pl  # import the functions from our pipeline
import sklearn.preprocessing

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
os.getcwd()

'C:\\Users\\edwar.WJM-SONYLAPTOP\\Documents\\GitHub\\ML_spring_2019\\exercise five'

# Working with the project grant database

In [3]:
projects = pl.load_and_peek_at_data('raw data/projects_2012_2013.csv',
                                 date_vars=['date_posted', 'datefullyfunded'],
                                 summary=True)

************************
************************


Head of data:
                          projectid                    teacher_acctid  \
0  00001ccc0e81598c4bd86bacb94d7acb  96963218e74e10c3764a5cfb153e6fea   
1  0000fa3aa8f6649abab23615b546016d  2a578595fe351e7fce057e048c409b18   
2  000134f07d4b30140d63262c871748ff  26bd60377bdbffb53a644a16c5308e82   
3  0001f2d0b3827bba67cdbeaa248b832d  15d900805d9d716c051c671827109f45   
4  0004536db996ba697ca72c9e058bfe69  400f8b82bb0143f6a40b217a517fe311   

                           schoolid  school_ncesid  school_latitude  \
0  9f3f9f2c2da7edda5648ccd10554ed8c   1.709930e+11        41.807654   
1  3432ed3d4466fac2f2ead83ab354e333   6.409801e+10        34.296596   
2  dc8dcb501c3b2bb0b10e9c6ee2cd8afd   6.227100e+10        34.078625   
3  8bea7e8c6e4279fca6276128db89292e   3.600090e+11        40.687286   
4  fbdefab6fe41e12c55886c610c110753   3.606870e+11        40.793018   

   school_longitude    school_city school_state school_metro  \
0   

## Look at graphs of our data

In [ ]:
pl.make_graphs(projects, normal_qq_plots=False)

In [ ]:
for imageName in glob2.glob('figures/*.png'):
    display(Image(filename=imageName))
    print(imageName)

In [4]:
# make dummy vars
cat_vars = ['primary_focus_subject', 'secondary_focus_area', 'secondary_focus_subject',
#             'school_city', 
            'school_metro', 
#             'school_state', 
            'school_charter', 'school_magnet',
           'primary_focus_area', 'resource_type', 'poverty_level', 'grade_level',
           'eligible_double_your_impact_match']
for var in cat_vars:
    projects = pl.make_dummies(projects, var)

In [5]:
# make dummy outcome var
projects['60_day_outcome'] = (projects['datefullyfunded'] -
                              projects['date_posted']) > pd.Timedelta('60 days')

In [6]:
# take a subset of our features
new_cat_vars = [col for col in projects if col.startswith('D_')]
vars_to_keep = new_cat_vars + ['date_posted', 'students_reached',
                               'total_price_including_optional_support', '60_day_outcome']
projects = projects[vars_to_keep]

# Make test and training sets

In [7]:
test_set_ranges = [('2012-01-01', '2012-04-30', '2012-07-01', '2012-10-31'), 
                   ('2012-01-01', '2012-10-31', '2013-01-01', '2013-04-30'), 
                   ('2012-01-01', '2013-04-30', '2013-07-01', '2013-10-31')]

In [8]:
date_subset = test_set_ranges[0]
train, test = pl.split_using_date(projects[0:40000],
                               train_start_date=date_subset[0],
                               train_end_date=date_subset[1],
                               test_start_date=date_subset[2],
                               test_end_date=date_subset[3])
print('training start and end dates are ', date_subset)
print('training data dimensions:')
print(train.shape)

# Fill missing values
pl.fill_missing(train, imputation_method='median')
pl.fill_missing(test, imputation_method='median')

# Split our training and test sets into x and y variables:
x = train.drop(['60_day_outcome', 'date_posted'], axis=1)
x_t = test.drop(['60_day_outcome', 'date_posted'], axis=1)
scaling = sklearn.preprocessing.MinMaxScaler(feature_range=(-1,1)).fit(x)
x_scaled = scaling.transform(x)
scaling = sklearn.preprocessing.MinMaxScaler(feature_range=(-1,1)).fit(x_t)
x_t_scaled = scaling.transform(x_t)
y = train['60_day_outcome']
y_t = test['60_day_outcome']

print('mean of 60 day outcome in training set:')
print(round(y.mean(), 2), '\n\n\n')


training start and end dates are  ('2012-01-01', '2012-04-30', '2012-07-01', '2012-10-31')
training data dimensions:
(6740, 95)
mean of 60 day outcome in training set:
0.31 





In [ ]:
pl.compare_models(x_data=x, x_data_scaled=x_scaled, y_data=y,
               x_test=x_t, x_test_scaled=x_t_scaled,
               y_test=y_t, use_test_data=True,
               run_svm=True,
               run_tree=True,
               run_boosted=False, 
               run_bagging=True,
               run_logit=True,
               run_a_forest=True,
               run_knn=True,
               mythresholds=[0, 1, 2, 5, 10, 20, 30, 50, 60, 70, 90, 100])

## Run our first set, with training set = first 6 months of data

In [ ]:
compare_models(x_data=x, x_data_scaled=x_scaled, y_data=y,
               x_test=x_t, x_test_scaled=x_t_scaled,
               y_test=y_t, use_test_data=True,
               #run_svm=True,
               run_tree=True,
               run_boosted=False, 
               run_bagging=True,
               run_logit=True,
               run_a_forest=True,
               run_knn=True,
                mythresholds=[0, .01, .02, .05, .1, .2, .3, .5, .6, .7, .9, 1])


## Run our second set, with training set = first 12 months of data

In [ ]:
date_subset = test_set_ranges[1]
train, test = split_using_date(projects,
                               train_start_date=date_subset[0],
                               train_end_date=date_subset[1])
print('training start and end dates are ', date_subset)
print('training data dimensions:')
print(train.shape)
print('\n\n')

x = train.drop(['60_day_outcome', 'date_posted'], axis=1)
x_t = test.drop(['60_day_outcome', 'date_posted'], axis=1)
scaling = MinMaxScaler(feature_range=(-1,1)).fit(x)
x_scaled = scaling.transform(x)
scaling = MinMaxScaler(feature_range=(-1,1)).fit(x_t)
x_t_scaled = scaling.transform(x_t)
y = train['60_day_outcome']
y_t = test['60_day_outcome']

print('mean of 60 day outcome in training set:')
print(round(y.mean(), 2), '\n')

compare_models(x_data=x, y_data=y, x_test=x_t, y_test=y_t, use_test_data=True,
               run_tree=True,
               run_boosted=False, 
               run_bagging=True,
               run_logit=True,
               run_a_forest=True,
               run_knn=True,
               mythresholds=[0, .01, .02, .05, .1, .2, .3, .5, .6, .7, .9, 1])

## Run our third set, with training set = first 18 months of data

In [ ]:
date_subset = test_set_ranges[2]
train, test = split_using_date(projects,
                               train_start_date=date_subset[0],
                               train_end_date=date_subset[1])
print('training start and end dates are ', date_subset)
print('training data dimensions:')
print(train.shape)

x = train.drop(['60_day_outcome', 'date_posted'], axis=1)
x_t = test.drop(['60_day_outcome', 'date_posted'], axis=1)
scaling = MinMaxScaler(feature_range=(-1,1)).fit(x)
x_scaled = scaling.transform(x)
scaling = MinMaxScaler(feature_range=(-1,1)).fit(x_t)
x_t_scaled = scaling.transform(x_t)
y = train['60_day_outcome']
y_t = test['60_day_outcome']

print('mean of 60 day outcome in training set:')
print(round(y.mean(), 2), '\n')

compare_models(x_data=x, y_data=y, x_test=x_t, y_test=y_t, use_test_data=True,
               run_tree=True,
               run_boosted=False, 
               run_bagging=True,
               run_logit=True,
               run_a_forest=True,
               run_knn=True,
               mythresholds=[0, .01, .02, .05, .1, .2, .3, .5, .6, .7, .9, 1])